# Content-based Filtering

Aproach:
- Table with features erstellen, ratings in zweitem table

- sklearn-kNN mit minkowski-distance auf features anwenden, erhalte matrix mit allen Nachbarn eines jeden Films inklusive Abstand

- Wähle für jeden Nutzer und Film die x-nächsten Nachbarn aus, die der Nutzer bewertet hat

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
kwargs = dict(random_state=42)

In [2]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

In [3]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [4]:
ratings.head()

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0


In [5]:
omdb.head()

,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,imdbID,...,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,Toy Story,1995.0,G,81.0,"John Lasseter (original story by), Pete Docter...",A cowboy doll is profoundly threatened and jea...,['English'],8.3,852896.0,tt0114709,...,0,4.0,11.0,22.0,0.0,1,0.0,3.0,0.0,0.0
1,Jumanji,1995.0,PG,104.0,"Jonathan Hensleigh (screenplay by), Greg Taylo...",When two kids find and play a magical board ga...,"['English', ' French']",7.0,294340.0,tt0113497,...,0,1.0,12.0,15.0,1.0,2,0.0,0.0,0.0,0.0
2,Grumpy Old Men,1993.0,PG-13,103.0,Mark Steven Johnson,A lifelong feud between two neighbors since ch...,['English'],7.0,41401.0,tt0107050,...,0,1.0,12.0,25.0,2.0,1,0.0,0.0,0.0,0.0
3,Waiting to Exhale,1995.0,R,124.0,"Terry McMillan (novel), Terry McMillan (screen...","Based on Terry McMillan's novel, this film fol...",['English'],5.9,9222.0,tt0114885,...,0,1.0,12.0,22.0,3.0,1,0.0,0.0,0.0,0.0
4,Father of the Bride Part II,1995.0,PG,106.0,"Albert Hackett (screenplay ""Father's Little Di...",George Banks must deal not only with the pregn...,['English'],6.0,33005.0,tt0113041,...,0,1.0,12.0,8.0,1.0,1,0.0,0.0,0.0,1.0


In [6]:
omdb.columns

Index(['Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated'],
      dtype='object')

In [7]:
# Select relevant columns
omdb = omdb[['imdbID', 'Year', 'Runtime', 'Language', 'imdbRating', 'imdbVotes', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_month', 'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated']]

In [8]:
omdb.isna().sum()

imdbID                       0
Year                         0
Runtime                      9
Language                     0
imdbRating                   4
imdbVotes                    5
Rotten Tomatoes           1237
Metacritic                4019
Series                       0
Released_month              56
PG_Rating                 2314
Available_languages          0
Oscars_won                   0
Oscars_nominated             0
Golden_globe_won             0
Golden_globe_nominated       0
dtype: int64

In [9]:
# Add movies which could not be retrieved by omdb
imdb_ids = pd.DataFrame(movies['imdbID'].unique()).rename(columns={0:'imdbID'})
omdb = imdb_ids.merge(omdb, how='left', on='imdbID')

In [10]:
# missing entries in omdb dataset
omdb[omdb['Year'].isna()]['imdbID']

707     tt1233247
2111    tt0800175
2241    tt0134948
3399    tt0257001
3810    tt0266860
4822    tt0720339
5118    tt0313487
7581    tt0439663
8140    tt0383974
9411    tt0182989
Name: imdbID, dtype: object

In [11]:
omdb.isna().sum()

imdbID                       0
Year                        10
Runtime                     15
Language                    10
imdbRating                  10
imdbVotes                   11
Rotten Tomatoes           1246
Metacritic                4025
Series                      10
Released_month              62
PG_Rating                 2320
Available_languages         10
Oscars_won                  10
Oscars_nominated            10
Golden_globe_won            10
Golden_globe_nominated      10
dtype: int64

In [12]:
# Replace NaN with 0 and handle accordingly for awards
for i in range (9,13):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(0)

In [13]:
omdb.loc[:,'Series'] = omdb.loc[:,'Series'].fillna(0)


In [14]:
# Comment Max: Vorschlag - Stattdessen Median/Modalwert benutzen benutzen
# Fill NaN Series
omdb.loc[:,'Series'] = omdb.loc[:,'Series'].fillna(0)
# Fill NaN Runtime
omdb.loc[:,'Runtime'] = omdb.loc[:,'Runtime'].fillna(omdb.loc[:,'Runtime'].median())
# Fill median for all others
for i in range (4,16):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(omdb.iloc[:,i].median())

In [15]:
# possibility to encode language as one-hot encoding
#mlb = MultiLabelBinarizer()
#mlb.fit_transform(merged_data['Language'])

In [16]:
# convert imdbID from string to float
omdb['imdbID'] = omdb['imdbID'].str.replace(r'tt','')
omdb['imdbID'] = omdb['imdbID'].astype(float)
omdb['imdbID'].unique()
ratings['imdbID'] = ratings['imdbID'].str.replace(r'tt','')
ratings['imdbID'] = ratings['imdbID'].astype(float)

In [17]:
# delete language for the moment as not yet encoded
omdb = omdb.drop(columns={'Language'})

In [18]:
# fill NaN of the Year
omdb['Year'] = omdb['Year'].fillna(omdb['Year'].median())

In [19]:
# show first entry
omdb.loc[omdb['imdbID'] == 114709]

,imdbID,Year,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,114709.0,1995.0,81.0,8.3,852896.0,10.0,9.5,0.0,11.0,0.0,1.0,0.0,3.0,0.0,0.0


In [20]:
omdb.sort_values('imdbID').iloc[62,:]

imdbID                    20530.0
Year                       1929.0
Runtime                      16.0
imdbRating                    7.7
imdbVotes                 43871.0
Rotten Tomatoes              10.0
Metacritic                    5.9
Series                        0.0
Released_month                6.0
PG_Rating                     0.0
Available_languages           1.0
Oscars_won                    0.0
Oscars_nominated              0.0
Golden_globe_won              0.0
Golden_globe_nominated        0.0
Name: 7536, dtype: float64

In [21]:
# set index as imdbID in ascending order
#features = omdb.set_index('imdbID', drop = True)
#features = features.sort_index()

In [22]:
features = omdb

In [23]:
# Standardize the features
scaler = preprocessing.StandardScaler()
features = scaler.fit_transform(features)

In [24]:
features

array([[-0.40529431,  0.37886077, -0.99135219, ...,  3.68058431,
        -0.06819758, -0.16894619],
       [-0.41125947,  0.37886077, -0.04121391, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [-0.44298994,  0.27105774, -0.08252427, ..., -0.27359947,
        -0.06819758, -0.16894619],
       ...,
       [-0.84453528, -2.90913151, -0.49562787, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [ 4.0735215 ,  1.07958043, -0.53693823, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [ 0.07639669,  0.59446682, -2.97424948, ..., -0.27359947,
        -0.06819758, -0.16894619]])

In [25]:
# features of the first entry (imdbID 439)
features[0]

array([-0.40529431,  0.37886077, -0.99135219,  1.68821913,  6.82544447,
        1.4084073 ,  2.62822861, -0.0807199 ,  1.26788359, -1.33063277,
       -0.52609534, -0.20232346,  3.68058431, -0.06819758, -0.16894619])

In [26]:
features.shape[0]

9423

In [27]:
# create NearesNeighbor model with the minkowski distance
knn = NearestNeighbors(metric='minkowski', p=2, algorithm='brute', n_jobs=-1)

In [28]:
# fit to all the features
knn.fit(features)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

In [30]:
# create two array for distance and index to store
distances = np.empty((features.shape[0],features.shape[0]))
indices = np.empty((features.shape[0],features.shape[0]))

# iterate over all imdbIDs
for i in range(0, features.shape[0]):
    # get all neighbors and the distances for each imdbID i
    distances[i], indices[i] = knn.kneighbors([features[i]], n_neighbors=features.shape[0])
    
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [31]:
distances

array([[0.00000000e+00, 1.96588178e+00, 3.43476464e+00, ...,
        2.76537266e+01, 3.00939977e+01, 5.11368348e+01],
       [0.00000000e+00, 1.38212405e+00, 1.50351137e+00, ...,
        2.60586696e+01, 2.90000512e+01, 5.01867989e+01],
       [2.10734243e-08, 5.99194841e-01, 6.26245488e-01, ...,
        2.62639064e+01, 2.87775021e+01, 5.01684724e+01],
       ...,
       [0.00000000e+00, 4.73830252e-01, 5.57099486e-01, ...,
        2.70022490e+01, 2.90429548e+01, 5.05102478e+01],
       [5.96046448e-08, 8.40906513e-01, 9.05067368e-01, ...,
        2.69628453e+01, 2.85531815e+01, 5.03836549e+01],
       [0.00000000e+00, 7.40186730e-01, 9.45900558e-01, ...,
        2.86539889e+01, 2.89338556e+01, 5.08908690e+01]])

In [32]:
indices[0]

array([   0., 1496., 1085., ..., 3796., 9019., 8322.])

In [104]:
index = omdb.loc[(omdb['imdbID'] == 47034.0)].index
print(index)   

Int64Index([2103], dtype='int64')


In [123]:
 index = omdb.loc[(omdb['imdbID'] == 47034.0)].index

In [124]:
movie_row = indices[[index]]
print(movie_row)

[[2103.  920. 7185. ... 3796. 9019. 8322.]]


/Users/FlorianGrabe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [135]:
rated_id = ratings.loc[ratings['user_id'] == 123]
print(rated_id['imdbID'])

2700      107688.0
20972     319061.0
51933     118715.0
69225     119822.0
91942     338135.0
            ...   
742419    248845.0
745967    112281.0
768099    108052.0
779240    181875.0
780940    110912.0
Name: imdbID, Length: 68, dtype: float64


In [136]:
 rated = np.empty((len(rated_id), 1))

In [137]:
 i = 0
for imdb in rated_id['imdbID']:
        #print(imdb)
    rated[i] = omdb.loc[(omdb['imdbID'] == imdb)].index
    i = i+1
        
print(rated.shape)

(68, 1)


In [138]:
print('Before: '+str(movie_row))
movie_frame = pd.DataFrame(movie_row)
print('After: '+str(movie_frame))

Before: [[2103.  920. 7185. ... 3796. 9019. 8322.]]
After:      0      1       2       3       4       5       6       7       8     \
0  2103.0  920.0  7185.0  6877.0  5827.0  8008.0  6892.0  1842.0  2675.0   

     9     ...    9413    9414   9415    9416    9417    9418    9419    9420  \
0  8026.0  ...  7799.0  6211.0  306.0  3481.0  1907.0  6162.0  7739.0  3796.0   

     9421    9422  
0  9019.0  8322.0  

[1 rows x 9423 columns]


In [139]:
movie_frame.columns = movie_frame.iloc[0]
movie_frame.drop(0, inplace=True)
print(movie_frame)

Empty DataFrame
Columns: [2103.0, 920.0, 7185.0, 6877.0, 5827.0, 8008.0, 6892.0, 1842.0, 2675.0, 8026.0, 7297.0, 2487.0, 3427.0, 699.0, 7236.0, 8418.0, 6509.0, 9288.0, 7285.0, 6563.0, 7312.0, 4616.0, 1960.0, 5775.0, 8393.0, 2722.0, 408.0, 988.0, 4473.0, 6990.0, 8038.0, 7108.0, 8167.0, 7263.0, 6421.0, 5793.0, 2602.0, 3260.0, 2375.0, 3515.0, 6771.0, 5992.0, 1800.0, 6661.0, 5097.0, 201.0, 2383.0, 5790.0, 3983.0, 5770.0, 3971.0, 5640.0, 870.0, 2436.0, 4526.0, 3956.0, 1944.0, 7921.0, 8352.0, 5813.0, 7573.0, 5468.0, 7623.0, 7305.0, 5433.0, 6497.0, 7568.0, 7006.0, 7310.0, 7264.0, 4851.0, 8007.0, 9170.0, 4726.0, 7607.0, 1933.0, 9212.0, 5654.0, 5382.0, 3158.0, 2959.0, 6704.0, 5409.0, 2368.0, 6274.0, 6377.0, 974.0, 7938.0, 3378.0, 7565.0, 5948.0, 8551.0, 2257.0, 181.0, 773.0, 1898.0, 858.0, 7187.0, 4664.0, 6585.0, ...]
Index: []

[0 rows x 9423 columns]


In [140]:
movie_frame.loc[1,:] = range(0,len(indices))
print(movie_frame)

0  2103.0  920.0   7185.0  6877.0  5827.0  8008.0  6892.0  1842.0  2675.0  \
1     0.0     1.0     2.0     3.0     4.0     5.0     6.0     7.0     8.0   

0  8026.0  ...  7799.0  6211.0  306.0   3481.0  1907.0  6162.0  7739.0  \
1     9.0  ...  9413.0  9414.0  9415.0  9416.0  9417.0  9418.0  9419.0   

0  3796.0  9019.0  8322.0  
1  9420.0  9421.0  9422.0  

[1 rows x 9423 columns]


In [141]:
rated_neighbors = np.empty((68,2))
for j in range(0, len(rated)):
    rated_neighbors[j] = movie_frame[rated[j]]
    if j % 10 ==0:
        print (movie_frame[rated[j]])
   

print(rated_neighbors)

0   531.0
1  3085.0
0   394.0
1  3043.0
0  3804.0
1  5820.0
0  1238.0
1  8050.0
0   106.0
1  9090.0
0   124.0
1  6120.0
0  1102.0
1  9387.0
[[3085. 3085.]
 [7128. 7128.]
 [8900. 8900.]
 [7169. 7169.]
 [4638. 4638.]
 [4666. 4666.]
 [5341. 5341.]
 [6374. 6374.]
 [3241. 3241.]
 [9372. 9372.]
 [3043. 3043.]
 [8430. 8430.]
 [9022. 9022.]
 [1711. 1711.]
 [3238. 3238.]
 [3632. 3632.]
 [5926. 5926.]
 [5155. 5155.]
 [7969. 7969.]
 [9173. 9173.]
 [5820. 5820.]
 [5002. 5002.]
 [6037. 6037.]
 [5362. 5362.]
 [ 530.  530.]
 [9009. 9009.]
 [2911. 2911.]
 [6086. 6086.]
 [4955. 4955.]
 [1075. 1075.]
 [8050. 8050.]
 [4147. 4147.]
 [9124. 9124.]
 [8975. 8975.]
 [6909. 6909.]
 [3874. 3874.]
 [9384. 9384.]
 [9049. 9049.]
 [4460. 4460.]
 [5830. 5830.]
 [9090. 9090.]
 [7015. 7015.]
 [6714. 6714.]
 [8709. 8709.]
 [8355. 8355.]
 [6723. 6723.]
 [8786. 8786.]
 [7539. 7539.]
 [5280. 5280.]
 [9037. 9037.]
 [6120. 6120.]
 [3056. 3056.]
 [6091. 6091.]
 [8014. 8014.]
 [3967. 3967.]
 [7364. 7364.]
 [3234. 3234.]
 [646

In [132]:
def predict_movie_rating(userID, imdbID):
        
    #imdbID den Indices zuordnen
    index = omdb.loc[(omdb['imdbID'] == imdbID)].index
    #indices aller nachbarn von nächsten Nachbarn zum letzten nachbarn
    movie_row = indices[[index]]
    print(movie_row)
    #Entstehendes Dataframe für Filme Filtern, die der User bewertet hat (ersten 20 nehmen)
    #imdbIDs die der User bewertet hat
    rated_id = ratings.loc[ratings['user_id'] == userID]
    print(rated_id['imdbID'])
    rated = np.empty((len(rated_id), 1))
    
    #Zuordnung der imbdID  zu dem Indexen der von dem User bewerteten filme
    i = 0
    for imdb in rated_id['imdbID']:
        #print(imdb)
        rated[i] = omdb.loc[(omdb['imdbID'] == imdb)].index
        i = i+1
        
    print(rated.shape)
    
    # Suche ersten 20 bewertete Nachbarfilme in Nachbarschaft
  
    movie_frame = pd.DataFrame(movie_row)

   
    movie_frame.columns = movie_frame.iloc[0]
    movie_frame.loc[0] = np.arange(0,len(indices))
    print(movie_frame)
    
    #print(len(neighbor_idx))
    #print(neighbor_idx)
    #auf basis des abstandes dieser Nachbarn ein rating vorhersagen
    

In [42]:
predict_movie_rating(123, 145653.0)

[[ 124.  142. 2849. ... 3796. 9019. 8322.]]
2700      107688.0
20972     319061.0
51933     118715.0
69225     119822.0
91942     338135.0
            ...   
742419    248845.0
745967    112281.0
768099    108052.0
779240    181875.0
780940    110912.0
Name: imdbID, Length: 68, dtype: float64
(68, 1)
0  0.0     1.0     2.0     3.0     4.0     5.0     6.0     7.0     8.0     \
0     0.0     1.0     2.0     3.0     4.0     5.0     6.0     7.0     8.0   

0  9.0     ...  9413.0  9414.0  9415.0  9416.0  9417.0  9418.0  9419.0  \
0     9.0  ...  9413.0  9414.0  9415.0  9416.0  9417.0  9418.0  9419.0   

0  9420.0  9421.0  9422.0  
0  9420.0  9421.0  9422.0  

[1 rows x 9423 columns]


/Users/FlorianGrabe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


# Bis hier hin aktuell

In [ ]:
#function that computes the k-nearest neighbors of each movie based on the features
for i in omdb['imdbID'].unique():
    try:
        distances , indices = knn.kneighbors(omdb.loc[omdb['imdbID'] == i], n_neighbors=10)
    except ValueError:
        print('Ohoh, ValueError')

In [37]:
user_ratings['imdbID'] = user_ratings['imdbID'].astype(float)

In [38]:
user_ratings

,index,imdbID,user_id,rating,Year,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,0,47034.0,1264,3.5,1954.0,96.0,7.6,27485.0,9.3,7.8,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,304141.0,213,2.5,2004.0,142.0,7.9,524511.0,9.0,8.2,0.0,6.0,1.0,1.0,0.0,2.0,0.0,0.0
2,2,369436.0,593,3.0,2008.0,88.0,5.7,61335.0,2.4,4.1,0.0,11.0,2.0,1.0,0.0,0.0,0.0,0.0
3,3,1077258.0,609,4.0,2007.0,105.0,7.1,196149.0,7.4,5.9,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,52182.0,1590,4.0,1958.0,100.0,7.4,6337.0,6.9,5.9,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787267,787267,83530.0,1032,3.0,1982.0,85.0,6.2,47327.0,4.2,4.8,0.0,12.0,1.0,1.0,0.0,0.0,0.0,0.0
787268,787268,107798.0,99,3.0,1993.0,141.0,6.6,73544.0,5.3,5.0,0.0,12.0,2.0,1.0,0.0,0.0,0.0,0.0
787269,787269,93857.0,333,3.0,1987.0,88.0,5.1,12882.0,0.7,2.8,0.0,7.0,2.0,2.0,0.0,0.0,0.0,0.0
787270,787270,144168.0,49,3.0,1999.0,94.0,6.1,17926.0,7.1,6.5,0.0,8.0,2.0,1.0,0.0,0.0,0.0,0.0


In [39]:
user_ratings.dtypes

index                       int64
imdbID                    float64
user_id                     int64
rating                    float64
Year                      float64
Runtime                   float64
imdbRating                float64
imdbVotes                 float64
Rotten Tomatoes           float64
Metacritic                float64
Series                    float64
Released_month            float64
PG_Rating                 float64
Available_languages       float64
Oscars_won                float64
Oscars_nominated          float64
Golden_globe_won          float64
Golden_globe_nominated    float64
dtype: object

In [50]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    pred = np.mean(neighbors['rating'])
    
    return pred

MemoryError: Unable to allocate 426. MiB for an array with shape (71, 787272) and data type float64

In [30]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    pred = np.mean(neighbors['rating'])
    
    return pred

In [37]:
## function that predicts the rating of a movie from its imdbID and its 20 nearest neighbors
# calculate similarity not based on user ratings? (only movie features)

def predict_movie_rating(imdbID, userID, user_data):
    # get 20 (or x) nearest neighbors that were rated by userID
    # TODO: Use function from kNN.ipynb to find optimal k for algorithm 
    k_neighbors = 10    
    
    # Select all ratings given by User #userID
    ratings = user_data.loc[user_data['user_id'] == userID]
    
    # Get real rating -> remove this in the end -> currently done for validation
    real_ratings = ratings.loc[(ratings['imdbID'] == imdbID)]
    
    #remove real rating
    ratings = ratings[ratings['imdbID'] != imdbID] 
     
    
    if (ratings.to_numpy().size>0):   
        
        # TODO:
        # Convert to Sparse Matrix Format
        #ratings_as_csr = csr_matrix(user_ratings)
        
        # Set algorithm and params 
        # TODO adapt neighbors based on number of given ratings by user_id ?
        knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

        # Training
        print('---- Training ConBF-kNN-Algorithm ----')
        print('user_id: '+str(userID))
        print('imdbID: '+str(imdbID))
        knn.fit(ratings)

        
        #Prediction -> get x nearest neighbors of imdbID
        distances , indices = knn.kneighbors(user_data.loc[user_data['imdbID'] == imdbID], n_neighbors=k_neighbors)
        
       
        # Collect neighbors by indices
        for i in range(0,indices.shape[1]):
            neighbors = ratings.iloc[[indices[0,i]]]
            
        # compute rating of movie(imbdID) based on the rating of the 20 nearest neighbors
        pred = compute_rating(neighbors, distances)
        
        #Generate Output for Understandability
        print('Predicted Rating for '+str(imdbID)+': '+str(pred))
        print('Real Rating of '+str(imdbID)+' was: '+ str(real_ratings['rating'].values[0]))
        
        # return rating prediction and real rating
        return pred , real_ratings['rating'].values[0]
        
    else:
         return "User has not rated other movies. Check input"
    

In [35]:
# Larger Test:
# TODO Refactoring needed #UglyPythonCode

def test_predict_mr(no_test_samples):
    # Sampling #no_test_samples of random samples from dataset for testing
    test_set = user_ratings.sample(n=no_test_samples)
    
    predictions = pd.DataFrame(columns=['Prediction'])
    reals = pd.DataFrame(columns=['Real_Rating'])
    
    # Iterate over test-set and generate predicitons for it
    # TODO get rid of ugly for-loop
    for row in test_set.itertuples():
        imdbID = row.imdbID
        userID = row.user_id
        pred , real = predict_movie_rating(imdbID, userID, user_ratings)
        predictions.loc[row.index] = pred
        reals.loc[row.index] = real
    
    rmse = mean_squared_error(reals['Real_Rating'], predictions['Prediction'], squared=False)
    print('RMSE: '+str(rmse))

In [40]:
n = 80
test_predict_mr(n)

---- Training ConBF-kNN-Algorithm ----
user_id: 376.0
imdbID: 120780.0
Predicted Rating for 120780.0: 1.0
Real Rating of 120780.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
user_id: 1431.0
imdbID: 94721.0
Predicted Rating for 94721.0: 4.0
Real Rating of 94721.0 was: 4.5
---- Training ConBF-kNN-Algorithm ----
user_id: 337.0
imdbID: 190641.0
Predicted Rating for 190641.0: 2.5
Real Rating of 190641.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
user_id: 361.0
imdbID: 94336.0
Predicted Rating for 94336.0: 4.0
Real Rating of 94336.0 was: 4.5
---- Training ConBF-kNN-Algorithm ----
user_id: 340.0
imdbID: 95016.0
Predicted Rating for 95016.0: 3.5
Real Rating of 95016.0 was: 3.0
---- Training ConBF-kNN-Algorithm ----
user_id: 70.0
imdbID: 80684.0
Predicted Rating for 80684.0: 4.5
Real Rating of 80684.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
user_id: 488.0
imdbID: 117119.0
Predicted Rating for 117119.0: 4.5
Real Rating of 117119.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
